In [1]:
import os
os.environ['OMP_NUM_THREADS'] = "32"
os.environ['MKL_NUM_THREADS'] = "32"
os.environ['NVIDIA_TF32_OVERRIDE'] = "1"

In [2]:
import time
import logging
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torch.utils.data.sampler import SubsetRandomSampler
from torch.profiler import profile, record_function, ProfilerActivity

In [3]:
torch.backends.cudnn.benchmark = True
torch.set_num_threads(32)
torch.set_num_interop_threads(1)

In [4]:
def build_data_loader(data_dir, batch_size, random_seed=42, valid_size=0.1, shuffle=True, test=False):
    transform = transforms.Compose([transforms.ToTensor()])
    transform.crop_size=512
    transform.resize_size=512
    
    train_dataset = datasets.CIFAR10(root=data_dir, train=True, download=False, transform=transform)
    test_dataset = datasets.CIFAR10(root=data_dir, train=False, download=False, transform=transform)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=4, pin_memory=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)

    return (train_loader, test_loader)

In [5]:
def train(model, data_loader, num_epochs, criterion, optimizer, device):
    total_steps = len(train_loader)
    for epoch in range(num_epochs):
        start = time.time()
        for step, (images, labels) in enumerate(train_loader):  
            # Move tensors to the configured device
            images = images.to(device)
            labels = labels.to(device)
        
            # Forward pass
            outputs = model(images)
            outputs.to(device)
            loss = criterion(outputs, labels)
        
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #print ('Step [{}/{}], Loss: {:.4f}'.format(step+1, total_steps, loss.item()))
        end = time.time()
               
        print ('Epoch [{}/{}], Loss: {:.4f}, time: {} seconds'.format(epoch+1, num_epochs, loss.item(), int(end-start)))
            

In [6]:
def train_amp(model, data_loader, num_epochs, criterion, optimizer, device):
    scaler = torch.cuda.amp.GradScaler()
    total_steps = len(train_loader)
    for epoch in range(num_epochs):
        start = time.time()
        for step, (images, labels) in enumerate(train_loader): 
            # Move tensors to the configured device
            images = images.to(device)
            labels = labels.to(device)

            with torch.autocast(device_type=device, dtype=torch.float16, enabled=True):
                output = model(images).to(device)
                loss = criterion(output, labels)

            # Backward and optimize
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            #print ('Step [{}/{}], Loss: {:.4f}'.format(step+1, total_steps, loss.item()))
        end = time.time()
               
        print ('Epoch [{}/{}], Loss: {:.4f}, time: {} seconds'.format(epoch+1, num_epochs, loss.item(), int(end-start)))
            

In [7]:
def test(model, test_loader, device):
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

    print('Accuracy of the network on the {} test images: {} %'.format(10000, 100 * correct / total))  

In [8]:
# General parameters
data_dir = '/tmp'
device = "cuda"

# Hyperparameters
lr = 0.00001
weight_decay = 0.005
batch_size = 2048
num_epochs = 50
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam

# Dataset 
train_loader, test_loader = build_data_loader(data_dir=data_dir, batch_size=batch_size)

# Model
net = models.efficientnet_b7()
net.train()
model = net.to(device)

# Optimizer
optimizer = optimizer(model.parameters(), lr, weight_decay=weight_decay)

In [9]:
# Training
start = time.time()
train_amp(model, train_loader, num_epochs, criterion, optimizer, device)
end = time.time()
print('Training time: {} seconds'.format(int(end-start)))
test(model, test_loader, device)

Epoch [1/50], Loss: 7.1894, time: 30 seconds
Epoch [2/50], Loss: 7.1552, time: 9 seconds
Epoch [3/50], Loss: 7.1069, time: 9 seconds
Epoch [4/50], Loss: 7.0585, time: 9 seconds
Epoch [5/50], Loss: 7.0442, time: 9 seconds
Epoch [6/50], Loss: 7.0586, time: 9 seconds
Epoch [7/50], Loss: 6.9536, time: 9 seconds
Epoch [8/50], Loss: 6.9571, time: 9 seconds
Epoch [9/50], Loss: 6.9259, time: 9 seconds
Epoch [10/50], Loss: 6.9408, time: 9 seconds
Epoch [11/50], Loss: 6.9134, time: 9 seconds
Epoch [12/50], Loss: 6.8597, time: 9 seconds
Epoch [13/50], Loss: 6.9018, time: 9 seconds
Epoch [14/50], Loss: 6.8310, time: 9 seconds
Epoch [15/50], Loss: 6.7867, time: 9 seconds
Epoch [16/50], Loss: 6.8043, time: 9 seconds
Epoch [17/50], Loss: 6.8440, time: 9 seconds
Epoch [18/50], Loss: 6.7893, time: 9 seconds
Epoch [19/50], Loss: 6.7008, time: 9 seconds
Epoch [20/50], Loss: 6.6904, time: 9 seconds
Epoch [21/50], Loss: 6.6725, time: 9 seconds
Epoch [22/50], Loss: 6.6459, time: 9 seconds
Epoch [23/50], Los

In [10]:
'''
activities = [ProfilerActivity.CPU, ProfilerActivity.CUDA]
prof = profile(activities=activities)
        
input_sample, _ = next(iter(train_loader))
input_sample.to(device)

prof.start()
train(model, train_loader, num_epochs, criterion, optimizer, device)
prof.stop()

#prof.export_chrome_trace("./cnn_trace.json")
print(prof.key_averages().table(sort_by="self_cuda_time_total", row_limit=15))
'''

'\nactivities = [ProfilerActivity.CPU, ProfilerActivity.CUDA]\nprof = profile(activities=activities)\n        \ninput_sample, _ = next(iter(train_loader))\ninput_sample.to(device)\n\nprof.start()\ntrain(model, train_loader, num_epochs, criterion, optimizer, device)\nprof.stop()\n\n#prof.export_chrome_trace("./cnn_trace.json")\nprint(prof.key_averages().table(sort_by="self_cuda_time_total", row_limit=15))\n'